In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

In [ ]:
import pandas as pd
import time
import numpy as np
import plotly.graph_objs as go

# df = pd.read_csv(
#     '/Users/jeremyhudsonchan/Dropbox/Files/Boston_College_Courses/Thesis/Code/Undergraduate-Thesis/Preprocessing_Code/final/data/proportions.csv', low_memory=False)
# officers = pd.read_csv(
#     '/Users/jeremyhudsonchan/Dropbox/Files/Boston_College_Courses/Thesis/Code/Undergraduate-Thesis/Preprocessing_Code/final/data/perm_unique_officers.csv', low_memory=False)

df = pd.read_csv('/content/gdrive/MyDrive/Thesis_Data/proportions.csv')
officers = pd.read_csv('/content/gdrive/MyDrive/Thesis_Data/perm_unique_officers.csv')

In [ ]:
print(df.columns)
print(df.head())
print(officers.columns)

In [ ]:
# choose only OfficerID and Beat columns
prep_df = df[['OfficerID', 'Beat', 'Beat Count', 'Beat Proportion']]
prep_officers = officers[['OfficerID', 'Beat']]
# group by CRID
df_grouped = df[['CRID', 'OfficerID', 'Beat', 'Beat Count', 'Beat Proportion']].groupby('CRID')
# make df_grouped into a numpy array
np_grouped = df_grouped.apply(lambda x: x.to_numpy())
# make np_grouped into a numpy array
np_grouped = np_grouped.to_numpy()

In [ ]:
# make df into numpy
np_df = prep_df.to_numpy()
np_officers = prep_officers.to_numpy()
# make array dtype object float
np_df = np_df.astype(float)
np_officers = np_officers.astype(float)

In [ ]:
# Column 0 is OfficerID, Column 1 is Beat
# ----------- Start of Original -----------
# get value counts of OfficerID
officer_counts = np.unique(np_df[:, 0], return_counts=True)
# get number of times each value appears in officer_counts
# sort in descending order
officer_counts = np.unique(officer_counts[1], return_counts=True)
# if number of complaints exceed 20, make all of the counts into a single bin
# make a copy of the original array to save for reference
officer_counts_copy = np.copy(officer_counts)
# this is to prevent the graph from being too stretched
for i in range(len(officer_counts[0])):
    if officer_counts[0][i] > 10:
        officer_counts[0][i] = 10
# plot density curve
fig = go.Figure()
fig.add_trace(go.Scatter(x=officer_counts[0], y=officer_counts[1], mode='lines', name='Data'))
fig.update_layout(title_text='Original Data')
fig.show()

In [ ]:
# print sum of counts
print('Sum of Counts: ', np.sum(officer_counts[1]))

In [ ]:
num_simulations = 1000

In [ ]:
# ----------- Start of Permutation Test -----------
# permutation test loop 1000 simulations
start_time = time.time()
# perm_test_results to be a 3d array
perm_test_results = []
for i in range(num_simulations):
    print(i)
    # make 1d array of 0s, length of np_officers
    perm_test = np.array([])
    # for each element in np_grouped, get length of each individual element
    for incident in np_grouped:
        # get length of each individual element
        num_officers = len(incident)
        # get beat of group
        beat = incident[0][2]
        if beat not in np_officers[:, 1]:
            continue
        # get officers in beat
        officers_in_beat = np_officers[np_officers[:, 1] == beat]
        if len(officers_in_beat) < num_officers:
            continue
        # randomly sample officers without replacement from officers in beat
        sample_officers = np.random.choice(officers_in_beat[:, 0], num_officers, replace=False)
        # add sample officers to perm_test
        perm_test = np.append(perm_test, sample_officers)
    # get value counts of OfficerID in perm_test
    perm_test_counts = np.unique(perm_test, return_counts=True)
    # print(perm_test_counts)
    # get number of times each value appears in perm_test_counts
    # sort in descending order
    perm_test_counts = np.unique(perm_test_counts[1], return_counts=True)
    # add perm_test_counts to perm_test_results
    perm_test_results.append(perm_test_counts)
end_time = time.time()
    

In [ ]:
print(end_time - start_time)
# print(perm_test_results)

In [ ]:
# make perm_test_results into a pandas df
perm_test_results = pd.DataFrame(perm_test_results)
# rename columns
perm_test_results.columns = ['Number of Complaints', 'Counts']
# get proportion of each number of complaints by going through each value of number of complaints one by one

In [ ]:
perm_test_results.head(10)

In [ ]:
# add 0 to each datapoint in Number of Complaints
perm_test_results['Number of Complaints'] = perm_test_results['Number of Complaints'].apply(lambda x: np.concatenate(([0], x)))
# and insert the np.sum(officer_counts[1]) - np.sum(polya_urn_results['Counts']) into the second column
perm_test_results['Counts'] = perm_test_results['Counts'].apply(lambda x: np.concatenate(([np.sum(officer_counts[1]) - np.sum(x)], x)))
# add 1 to all the counts to perm_test_results
perm_test_results['Number of Complaints'] = perm_test_results['Number of Complaints'].apply(lambda x: x + 1)

In [ ]:
perm_test_results.head(10)

In [ ]:
# for each row in pem_test_results, print out sum of counts column
perm_sum = []
for i in range(len(perm_test_results)):
    perm_sum.append(sum(perm_test_results.iloc[i, 1]))

print(np.mean(perm_sum))

In [ ]:
# explode the number of complaints column, then get max value
max_complaints = max(perm_test_results['Number of Complaints'].explode())
print(max_complaints)
# create dictionary of number of complaints and counts, key is number of complaints, value is counts
# keys should range from 1 to max_complaints
complaints_dict = {}
for i in range(1, 11):
    complaints_dict[i] = []
# for each row in perm_test_results, get the list of Number of Complaints, then get the list of Counts, then map them to the dictionary
for index, row in perm_test_results.iterrows():
    # get list of number of complaints
    complaints = row['Number of Complaints']
    # get list of counts
    counts = row['Counts']
    # map them to the dictionary
    for i in range(len(complaints)):
        if complaints[i] < 10:
            complaints_dict[complaints[i]].append(counts[i])
        else:
            complaints_dict[10].append(counts[i])

# if the list length is not equal to num_simulations, then add 0s to the list until it is equal to num_simulations
for key in complaints_dict:
    if len(complaints_dict[key]) != num_simulations:
        complaints_dict[key] = complaints_dict[key] + [0] * (num_simulations - len(complaints_dict[key]))

In [ ]:
# get the average of each list in the dictionary
perm_avg_complaints_dict = {}
for key in complaints_dict:
    perm_avg_complaints_dict[key] = sum(complaints_dict[key])/num_simulations

In [ ]:
# get the 95% confidence interval of each list in the dictionary
upper_perm_ci_complaints_dict = {}
lower_perm_ci_complaints_dict = {}
for key in complaints_dict:
    # 95% confidence interval using quantile
    upper_perm_ci_complaints_dict[key] = np.quantile(complaints_dict[key], 0.975)
    lower_perm_ci_complaints_dict[key] = np.quantile(complaints_dict[key], 0.025)

In [ ]:
# plot curve
# Number of Complaints are on the x-axis, Counts are on the y-axis
fig = go.Figure()
fig.add_trace(go.Scatter(x=list(perm_avg_complaints_dict.keys()), y=list(perm_avg_complaints_dict.values()), mode='lines', name='Permutation Test Results'))
# add line with color
fig.add_trace(go.Scatter(x=list(upper_perm_ci_complaints_dict.keys()), y=list(upper_perm_ci_complaints_dict.values()), mode='lines', name='Permutation Test 95% Confidence Interval', line=dict(color='rgb(235, 52, 58)', dash='dash')))
fig.add_trace(go.Scatter(x=list(lower_perm_ci_complaints_dict.keys()), y=list(lower_perm_ci_complaints_dict.values()), mode='lines', name='Permutation Test 95% Confidence Interval', line=dict(color='rgb(235, 52, 58)', dash='dash'), fill='tonexty', fillcolor='rgba(235, 52, 58,0.2)'))
fig.add_trace(go.Scatter(x=officer_counts[0], y=officer_counts[1], mode='lines', name='Data'))
fig.update_layout(title_text='Permutation Test Results')
fig.show()